# Jackknife

The jackknife was proposed by M.H. Quenouille in 1949, and later refined and given its current name by John Tukey in 1956. It is similar to the bootstrap in that it involves resampling, but instead of sampling with replacement, the method samples without replacement.

## Procedure

**Definition:**  The jackknife samples are computed by leaving out one observation $X_i$ from $\vec{x} = (x_1, ..., x_n)$ at a time: 

A jackknife sample looks like

$$\vec{x}_{(i)} = (x_1, x_2, ..., x_{i-1}, x_{i+1}, ..., x_n)$$

The dimension of a jackknife sample is $m = n-1$. There are $n$ unique different jackknife samples (which is probably a limitation). 

Notice: **No sampling is needed to compute the $n$ jackknife samples!**

The $i$th Jackknife replicate $\hat{\theta}_{(i)}$ is defined as the value of the estimator or statistic $s(.)$ evaluated at the $i$th Jackknife sample $$\hat{\theta}_{(i)} = s(\vec{X}_{(i)}), \forall i = 1, ..., n.$$

**Example:** Let $\vec{X} = (1,5,7,10,12)^\prime$ and define $\hat{\theta} = s(\vec{X}) = \bar{X}$ (sample mean).

Step 1: The jackknife samples are 

$\vec{X}_{(1)} = (5,7,10,12)^\prime$

$\vec{X}_{(2)} = (3,7,10,12)^\prime$

$\vec{X}_{(3)} = (3,5,10,12)^\prime$

$\vec{X}_{(4)} = (3,5,7,12)^\prime$

$\vec{X}_{(5)} = (3,5,7,10)^\prime$

Notice: $n$ samples are generated, each of size $m=n-1$.

The 5 Jackknife replicates are: 

$\hat{\theta}_{(1)} = \frac{5+7+10+12}{4}$

$\hat{\theta}_{(2)} = \frac{3+7+10+12}{4}$

$\hat{\theta}_{(3)} = \frac{3+5+10+12}{4}$

$\hat{\theta}_{(4)} = \frac{3+5+7+12}{4}$

$\hat{\theta}_{(5)} = \frac{3+5+7+10}{4}$

## Standard error estimate

Step 1: Compute the $n$ jackknife samples (see above)

Step 2: Evlauate the $n$ Jackknife replications (see above)

Step 3: Evaluate standard error estimate as $se_{jack} = [\frac{n-1}{n} \sum(\hat{\theta}_{(.)} - \hat{\theta}_{(i)})^2]^{1/2}$

where $\hat{\theta}_{(.)} = \frac{1}{n} \sum_{i=1}^n \hat{\theta}_{(i)}$

## Bias estimate

Step 1: Compute the $n$ jackknife samples (see above)

Step 2: Evlauate the $n$ Jackknife replications (see above)

Step 3: Jackknife bias estimate is $bias_{jack} = (n-1) (\hat{\theta}_{.} - \hat{\theta})$ where $\hat{\theta}_{(.)} = \frac{1}{n} \sum_{i=1}^n \hat{\theta}_{(i)}$

In [13]:
import copy
import numpy as np

x = [3,5,7,10,12]
n = len(x)

replications = []
for i in range(len(x)):
    xi = copy.copy(x)
    # Remove current index's sample
    xi.pop(i)
    replication_i = sum(xi) / len(xi)
    replications.append(replication_i)

replications = np.array(replications)
theta_dot = (1./n) * np.sum(replications)
var = ( (n-1) / n ) * np.sum((theta_dot - replications)**2)
se = np.sqrt(var)
bias = (n-1) * (theta_dot - replications)

#print("Using python plugins", "variance:", np.var(replications), "std", np.std(replications))
"Standard error of jackknife:", se, "bias", bias

('Standard error of jackknife:',
 1.6309506430300091,
 'bias',
 array([-4.4, -2.4, -0.4,  2.6,  4.6]))

## Relationship with bootstrap

When $n$ is small, it is easier (faster) to compute the n-Jackknife replications. However, the Jackknife uses less information (less samples) than the bootstrap. In fact, the jackknife is an approximator to the bootstrap. 

## Failure of the Jackknife

The jackknife can faile if the estimate $\hat{\theta}$ is not smooth (i.e. a small change in the data can cause a large change in the statistic). An example of a simple non-smooth statistic is the median.

Example: $\vec{X} = (10,27,31,40,46,50,52,104,146)^\prime$

In [15]:
import copy
import numpy as np

x = [10,27,31,40,46,50,52,104,146]
n = len(x)

replications = []
for i in range(len(x)):
    xi = copy.copy(x)
    # Remove current index's sample
    xi.pop(i)
    replication_i = np.median(xi)
    replications.append(replication_i)

print(replications)
replications = np.array(replications)
theta_dot = (1./n) * np.sum(replications)
var = ( (n-1) / n ) * np.sum((theta_dot - replications)**2)
se = np.sqrt(var)
bias = (n-1) * (theta_dot - replications)

#print("Using python plugins", "variance:", np.var(replications), "std", np.std(replications))
"Standard error of jackknife:", se, "bias", bias

[48.0, 48.0, 48.0, 48.0, 45.0, 43.0, 43.0, 43.0, 43.0]


('Standard error of jackknife:',
 6.681465057054626,
 'bias',
 array([-20.44444444, -20.44444444, -20.44444444, -20.44444444,
          3.55555556,  19.55555556,  19.55555556,  19.55555556,
         19.55555556]))

There are many duplicates in the replications. This maps down nine statistics down to three, caused by the nonsmooth statistic. The jackknife is not a good estimation method for estimating percentiles (such as the median), or when using any other non-smooth estimator. 

To resolve this issue, the method called **Delete-d jackknife** was created.

## Delete-d Jackknife

The delete-d jackknife subsamples are computed by leaving out $d$ observations from $\vec{X}$ at a time. This is the essence of cross validation!

The dimension of the subsample is $n-d$. The number of possible subsamples now rises to $\begin{pmatrix}n\\d\\\end{pmatrix}$

**Algorithm**

Step 1: Compute all $\begin{pmatrix}n\\d\\\end{pmatrix}$ d-jackknife subsamples from $\vec{x}$.

Step 2: Evaluate the jackknife replications $\hat{\theta}_{(i)} = s(\vec{x}_{(i)})$

Step 3: Estimation of the s.e. (where $n=rd$, where $r$ is a constant)

$$\hat{se}_{d-jack} = \{ \frac{r}{\begin{pmatrix}n\\d\\\end{pmatrix}} \sum_i (\hat{\theta}_{(i)} - \hat{\theta}_{.})^2 \}^{1/2}$$

where $\hat{\theta}_{.} = \frac{\sum_i \theta_{(i)}}{\begin{pmatrix}n\\d\\\end{pmatrix}}$

The inconsistency of the jackknife subsamples with non-smooth statistics can be fixed using delete-d jackknife subsamples.
